In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 20)




## Raw Data Into DataFrames

In [2]:
# Define file paths - adjust this path to match your project location
# Replace this with the actual path to your work_supa project
project_path = Path(r"C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa")

parquet_files = {
    'universe': project_path / 'universe' / 'universe.parquet',
    'portfolio': project_path / 'portfolio' / 'portfolio.parquet',
    'combined_runs': project_path / 'runs' / 'combined_runs.parquet',
    'run_monitor': project_path / 'runs' / 'run_monitor.parquet',
    'bond_z': project_path / 'historical g spread' / 'bond_z.parquet'
}

print("📊 Pipeline Parquet Files to DataFrames")
print("=" * 50)
print("🔍 Checking parquet file availability:")
for name, path in parquet_files.items():
    status = "✅" if path.exists() else "❌"
    print(f"{status} {name}: {path}")
print()

# Load all parquet files into DataFrames
print("📥 Loading parquet files into DataFrames...")
print("=" * 50)

# Dictionary to store all DataFrames
dataframes = {}

for name, file_path in parquet_files.items():
    if file_path.exists():
        try:
            # Load the parquet file
            df = pd.read_parquet(file_path)
            
            # Store in dictionary
            dataframes[name] = df
            
            # Also create a variable with the same name as the file
            globals()[name] = df
            
            print(f"✅ {name}: {df.shape[0]:,} rows × {df.shape[1]} columns ({file_path.stat().st_size / 1024 / 1024:.1f} MB)")
            
        except Exception as e:
            print(f"❌ Error loading {name}: {e}")
    else:
        print(f"❌ File not found: {file_path}")

print(f"\n📊 Total DataFrames loaded: {len(dataframes)}")

# Simple .info() display for all DataFrames
print("📊 DATAFRAME INFO SUMMARY")
print("=" * 80)


# Simple .info() for all DataFrames
print("📊 DATAFRAME INFO")
print("=" * 80)

for name, df in dataframes.items():
    print(f"\n {name.upper()}")
    print("=" * 40)
    df.info()
    print()

📊 Pipeline Parquet Files to DataFrames
🔍 Checking parquet file availability:
✅ universe: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\universe\universe.parquet
✅ portfolio: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\portfolio\portfolio.parquet
✅ combined_runs: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\runs\combined_runs.parquet
✅ run_monitor: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\runs\run_monitor.parquet
✅ bond_z: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\historical g spread\bond_z.parquet

📥 Loading parquet files into DataFrames...
✅ universe: 38,917 rows × 47 columns (4.9 MB)
✅ portfolio: 9,390 rows × 51 columns (1.2 MB)
✅ co

## Creating The z_all_enhanced DataFrame

In [3]:
# Create z_all_enhanced DataFrame - OPTIMIZED VERSION
print("Creating z_all_enhanced DataFrame (OPTIMIZED)...")
print("=" * 60)

import time
start_time = time.time()

# Start with all columns from BOND_Z
z_all_enhanced = dataframes['bond_z'].copy()

# Get the last date from universe and portfolio
last_universe_date = dataframes['universe']['Date'].max()
last_portfolio_date = dataframes['portfolio']['Date'].max()

print(f"Last universe date: {last_universe_date}")
print(f"Last portfolio date: {last_portfolio_date}")

# Define UNIVERSE columns to join
universe_columns = ['Custom_Sector',
    'CPN TYPE', 'Ticker', 'G Sprd', 'Yrs (Cvn)', 'Currency', 
    'Equity Ticker', 'Yrs Since Issue', 'Risk', 'Rating', 
    'Yrs Since Issue Bucket', 'Yrs (Mat) Bucket'
]

# Define RUN_MONITOR columns to join
run_monitor_columns = [
    'Bid/Offer', 'Dealer @ Best Bid', 'Dealer @ Best Offer', 
    'Size @ Best Bid', 'Size @ Best Offer'
]

print(f"⚡ Starting with {len(z_all_enhanced):,} rows from BOND_Z")

# OPTIMIZATION 1: Pre-filter UNIVERSE and RUN_MONITOR to only needed columns and CUSIPs
print("\nPre-filtering data for faster joins...")

# Get unique CUSIPs from BOND_Z
bond_cusips = set(z_all_enhanced['CUSIP_1'].unique()) | set(z_all_enhanced['CUSIP_2'].unique())

# Filter UNIVERSE to only needed CUSIPs and columns, and only last date
universe_filtered = dataframes['universe'][
    (dataframes['universe']['CUSIP'].isin(bond_cusips)) &
    (dataframes['universe']['Date'] == last_universe_date)
][['CUSIP'] + universe_columns].copy()

# Filter RUN_MONITOR to only needed CUSIPs and columns  
run_monitor_filtered = dataframes['run_monitor'][
    dataframes['run_monitor']['CUSIP'].isin(bond_cusips)
][['CUSIP'] + run_monitor_columns].copy()

# Filter PORTFOLIO to only last date
portfolio_filtered = dataframes['portfolio'][
    dataframes['portfolio']['Date'] == last_portfolio_date
].copy()

print(f"✅ UNIVERSE filtered: {len(universe_filtered):,} rows (from {len(dataframes['universe']):,})")
print(f"✅ RUN_MONITOR filtered: {len(run_monitor_filtered):,} rows (from {len(dataframes['run_monitor']):,})")
print(f"✅ PORTFOLIO filtered: {len(portfolio_filtered):,} rows (from {len(dataframes['portfolio']):,})")

# OPTIMIZATION 2: Create lookup dictionaries for faster joins
print("\n🔧 Creating lookup dictionaries...")

# UNIVERSE lookup dictionaries
universe_lookup_1 = {}
universe_lookup_2 = {}
for _, row in universe_filtered.iterrows():
    cusip = row['CUSIP']
    universe_lookup_1[cusip] = {f"{col}_1": row[col] for col in universe_columns}
    universe_lookup_2[cusip] = {f"{col}_2": row[col] for col in universe_columns}

# RUN_MONITOR lookup dictionaries
run_monitor_lookup_1 = {}
run_monitor_lookup_2 = {}
for _, row in run_monitor_filtered.iterrows():
    cusip = row['CUSIP']
    run_monitor_lookup_1[cusip] = {f"{col}_1": row[col] for col in run_monitor_columns}
    run_monitor_lookup_2[cusip] = {f"{col}_2": row[col] for col in run_monitor_columns}

# PORTFOLIO CUSIP set for fast membership testing (only last date)
portfolio_cusips = set(portfolio_filtered['CUSIP'].unique())

print(f"✅ Lookup dictionaries created for {len(bond_cusips):,} unique CUSIPs")

# OPTIMIZATION 3: Vectorized operations for all joins
print("\n⚡ Performing vectorized joins...")

# Add UNIVERSE data for CUSIP_1
for col in universe_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in universe_lookup_1.items()}
    )

# Add UNIVERSE data for CUSIP_2  
for col in universe_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in universe_lookup_2.items()}
    )

# Add RUN_MONITOR data for CUSIP_1
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in run_monitor_lookup_1.items()}
    )

# Add RUN_MONITOR data for CUSIP_2
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in run_monitor_lookup_2.items()}
    )

# Add ownership columns (vectorized) - only for last portfolio date
z_all_enhanced['Own_CUSIP_1'] = z_all_enhanced['CUSIP_1'].isin(portfolio_cusips).astype(int)
z_all_enhanced['Own_CUSIP_2'] = z_all_enhanced['CUSIP_2'].isin(portfolio_cusips).astype(int)

# Performance summary
end_time = time.time()
execution_time = end_time - start_time

print("\n" + "=" * 60)
print("🎉 z_all_enhanced DataFrame created successfully!")
print(f"⚡ Execution time: {execution_time:.3f} seconds")
print(f"📊 Final shape: {z_all_enhanced.shape[0]:,} rows × {z_all_enhanced.shape[1]} columns")

# Quick validation
print("\nVALIDATION SUMMARY:")
print("=" * 30)
print(f"✅ CUSIP_1 UNIVERSE matches: {z_all_enhanced['Ticker_1'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ CUSIP_2 UNIVERSE matches: {z_all_enhanced['Ticker_2'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ CUSIP_1 RUN_MONITOR matches: {z_all_enhanced['Bid/Offer_1'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ CUSIP_2 RUN_MONITOR matches: {z_all_enhanced['Bid/Offer_2'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ Own_CUSIP_1 positions: {z_all_enhanced['Own_CUSIP_1'].sum():,}")
print(f"✅ Own_CUSIP_2 positions: {z_all_enhanced['Own_CUSIP_2'].sum():,}")

print(f"\n🚀 Performance: {len(z_all_enhanced):,} records processed in {execution_time:.3f}s")
print(f"⚡ Speed: {len(z_all_enhanced)/execution_time:,.0f} records/second")

Creating z_all_enhanced DataFrame (OPTIMIZED)...
Last universe date: 2025-07-16 00:00:00
Last portfolio date: 2025-06-30 00:00:00
⚡ Starting with 1,109,305 rows from BOND_Z

Pre-filtering data for faster joins...
✅ UNIVERSE filtered: 1,476 rows (from 38,917)
✅ RUN_MONITOR filtered: 4,450 rows (from 5,830)
✅ PORTFOLIO filtered: 654 rows (from 9,390)

🔧 Creating lookup dictionaries...
✅ Lookup dictionaries created for 1,490 unique CUSIPs

⚡ Performing vectorized joins...

🎉 z_all_enhanced DataFrame created successfully!
⚡ Execution time: 2.151 seconds
📊 Final shape: 1,109,305 rows × 47 columns

VALIDATION SUMMARY:
✅ CUSIP_1 UNIVERSE matches: 1,095,890/1,109,305
✅ CUSIP_2 UNIVERSE matches: 1,101,874/1,109,305
✅ CUSIP_1 RUN_MONITOR matches: 596,829/1,109,305
✅ CUSIP_2 RUN_MONITOR matches: 554,168/1,109,305
✅ Own_CUSIP_1 positions: 53,931
✅ Own_CUSIP_2 positions: 54,766

🚀 Performance: 1,109,305 records processed in 2.151s
⚡ Speed: 515,754 records/second


## Edits To z_all_enhanced

In [4]:
# Create z_all_enhanced DataFrame - OPTIMIZED VERSION (CLEAN)
# Start with all columns from BOND_Z
z_all_enhanced = dataframes['bond_z'].copy()

# Get the last date from universe and portfolio
last_universe_date = dataframes['universe']['Date'].max()
last_portfolio_date = dataframes['portfolio']['Date'].max()

print(f"Last universe date: {last_universe_date}")
print(f"Last portfolio date: {last_portfolio_date}")

# Define UNIVERSE columns to join
universe_columns = ['Custom_Sector',
    'CPN TYPE', 'Ticker', 'G Sprd', 'Yrs (Cvn)', 'Currency', 
    'Equity Ticker', 'Yrs Since Issue', 'Risk', 'Rating', 
    'Yrs Since Issue Bucket', 'Yrs (Mat) Bucket'
]

# Define RUN_MONITOR columns to join
run_monitor_columns = [
    'Bid/Offer', 'Dealer @ Best Bid', 'Dealer @ Best Offer', 
    'Size @ Best Bid', 'Size @ Best Offer'
]

# Get unique CUSIPs from BOND_Z
bond_cusips = set(z_all_enhanced['CUSIP_1'].unique()) | set(z_all_enhanced['CUSIP_2'].unique())

# Filter UNIVERSE to only needed CUSIPs and columns, and only last date
universe_filtered = dataframes['universe'][
    (dataframes['universe']['CUSIP'].isin(bond_cusips)) &
    (dataframes['universe']['Date'] == last_universe_date)
][['CUSIP'] + universe_columns].copy()

# Filter RUN_MONITOR to only needed CUSIPs and columns  
run_monitor_filtered = dataframes['run_monitor'][
    dataframes['run_monitor']['CUSIP'].isin(bond_cusips)
][['CUSIP'] + run_monitor_columns].copy()

# Filter PORTFOLIO to only last date
portfolio_filtered = dataframes['portfolio'][
    dataframes['portfolio']['Date'] == last_portfolio_date
].copy()

# Create lookup dictionaries for faster joins
universe_lookup_1 = {}
universe_lookup_2 = {}
for _, row in universe_filtered.iterrows():
    cusip = row['CUSIP']
    universe_lookup_1[cusip] = {f"{col}_1": row[col] for col in universe_columns}
    universe_lookup_2[cusip] = {f"{col}_2": row[col] for col in universe_columns}

run_monitor_lookup_1 = {}
run_monitor_lookup_2 = {}
for _, row in run_monitor_filtered.iterrows():
    cusip = row['CUSIP']
    run_monitor_lookup_1[cusip] = {f"{col}_1": row[col] for col in run_monitor_columns}
    run_monitor_lookup_2[cusip] = {f"{col}_2": row[col] for col in run_monitor_columns}

# PORTFOLIO CUSIP set for fast membership testing (only last date)
portfolio_cusips = set(portfolio_filtered['CUSIP'].unique())

# Add UNIVERSE data for CUSIP_1
for col in universe_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in universe_lookup_1.items()}
    )

# Add UNIVERSE data for CUSIP_2  
for col in universe_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in universe_lookup_2.items()}
    )

# Add RUN_MONITOR data for CUSIP_1
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in run_monitor_lookup_1.items()}
    )

# Add RUN_MONITOR data for CUSIP_2
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in run_monitor_lookup_2.items()}
    )

# Add ownership columns (vectorized) - only for last portfolio date
z_all_enhanced['Own_CUSIP_1'] = z_all_enhanced['CUSIP_1'].isin(portfolio_cusips).astype(int)
z_all_enhanced['Own_CUSIP_2'] = z_all_enhanced['CUSIP_2'].isin(portfolio_cusips).astype(int)

# Delete specified columns
columns_to_delete = [
    'Dealer @ Best Bid_1',
    'Size @ Best Bid_1', 
    'Dealer @ Best Offer_2',
    'Size @ Best Offer_2'
]
z_all_enhanced = z_all_enhanced.drop(columns=columns_to_delete)

# Add CR01 columns
risk_1_numeric = pd.to_numeric(z_all_enhanced['Risk_1'], errors='coerce')
risk_2_numeric = pd.to_numeric(z_all_enhanced['Risk_2'], errors='coerce')

z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] = (
    (z_all_enhanced['Size @ Best Offer_1'].fillna(0) / 1000000) * risk_1_numeric.fillna(0)
)

z_all_enhanced['CUSIP_2 CR01 @ Best Bid'] = (
    (z_all_enhanced['Size @ Best Bid_2'].fillna(0) / 1000000) * risk_2_numeric.fillna(0)
)

# Move CUSIP_1 and CUSIP_2 to the end
cols_without_cusips = [col for col in z_all_enhanced.columns if col not in ['CUSIP_1', 'CUSIP_2']]
z_all_enhanced = z_all_enhanced[cols_without_cusips + ['CUSIP_1', 'CUSIP_2']]

# Sort by Z_Score high to low (descending)
z_all_enhanced = z_all_enhanced.sort_values('Z_Score', ascending=False).reset_index(drop=True)

# Define numeric columns to round to 1 decimal (excluding CR01 and Size columns)
numeric_cols_1_decimal = [
    'Last_Spread', 'Z_Score', 'Max', 'Min', 'Last_vs_Max', 'Last_vs_Min', 'Percentile',
    'G Sprd_1', 'Yrs (Cvn)_1', 'Yrs Since Issue_1', 'G Sprd_2', 'Yrs (Cvn)_2', 'Yrs Since Issue_2',
    'Bid/Offer_1', 'Bid/Offer_2'
]

# Apply formatting within display() only
def format_for_display(df):
    """Format DataFrame for display without changing underlying data"""
    # Set display options to show full content
    pd.set_option('display.max_colwidth', None)  # Show full column content
    pd.set_option('display.width', None)  # Don't wrap wide tables
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.max_rows', None)  # Show all rows
    
    formatted_df = df.copy()
    
    # Round numeric columns to 1 decimal
    for col in numeric_cols_1_decimal:
        if col in formatted_df.columns:
            formatted_df[col] = formatted_df[col].round(1)
    
    # Format CR01 columns with thousand separators and no decimals
    formatted_df['CUSIP_1 CR01 @ Wide Offer'] = formatted_df['CUSIP_1 CR01 @ Wide Offer'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    formatted_df['CUSIP_2 CR01 @ Best Bid'] = formatted_df['CUSIP_2 CR01 @ Best Bid'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    
    # Format Size columns with thousand separators and no decimals
    formatted_df['Size @ Best Offer_1'] = formatted_df['Size @ Best Offer_1'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    formatted_df['Size @ Best Bid_2'] = formatted_df['Size @ Best Bid_2'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    
    # Format Risk columns with thousand separators and no decimals
    formatted_df['Risk_1'] = formatted_df['Risk_1'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    formatted_df['Risk_2'] = formatted_df['Risk_2'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    
    return formatted_df

display(format_for_display(z_all_enhanced.head(20)))

Last universe date: 2025-07-16 00:00:00
Last portfolio date: 2025-06-30 00:00:00


,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
0,HOMEQU 5.82 08/13/29,HOMEQU 6.552 10/18/27,51.0,5.2,51.0,26.0,0.0,25.1,99.8,Mortgage Provider,FIXED,HOMEQU,212.0,4.0,CAD,HEQ CN Equity,0.9,363,NR,0.50-1,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,3.0,RBC,"3,000,000",NaN,CIBC,"3,000,000",0,1,"1,089",658,43739CAC5,43739CAB7
1,METYSL 3.779 05/13/30,RCICN 3.8 03/15/32,84.4,4.8,84.4,41.6,0.0,42.8,99.8,Other,FIXED,METYSL,NaN,4.8,CAD,2405318D US Equity,5.2,,nan,5-7,4.1-5.1,Cable/Telco,FIXED,RCICN,148.6,6.7,USD,RCI/B CN Equity,3.3,534,BBB-,3-5,5.1-7.1,NaN,NaN,,NaN,NaN,,0,0,0,0,29279KAA6,775109CC3
2,FENPCL 2.501 07/31/38,RCICN 3.8 03/15/32,102.2,4.7,102.2,36.0,0.0,66.2,99.8,Other,FIXED,FENPCL,230.5,6.8,CAD,1738436D CN Equity,5.9,515,NR,5-7,10.1-25.1,Cable/Telco,FIXED,RCICN,148.6,6.7,USD,RCI/B CN Equity,3.3,534,BBB-,3-5,5.1-7.1,NaN,NaN,,NaN,NaN,,0,0,0,0,31446VAA6,775109CC3
3,NYLIFE 5 06/06/29,RCICN 3.8 03/15/32,-75.5,4.7,-75.2,-114.0,-0.3,38.5,99.4,Financial Maples,FIXED,NYLIFE,94.0,3.9,USD,2141Z US Equity,1.1,358,AA+,1-2,3.1-4.1,Cable/Telco,FIXED,RCICN,148.6,6.7,USD,RCI/B CN Equity,3.3,534,BBB-,3-5,5.1-7.1,NaN,NaN,,NaN,NaN,,0,0,0,0,64952WFG3,775109CC3
4,ENBGAS 5.46 10/06/28,HYDONE 1.69 01/16/31,23.4,4.6,23.4,3.3,0.0,20.1,99.8,Utility,FIXED,ENBGAS,48.8,3.1,CAD,ENB CN Equity,1.8,307,A-,1-2,3.1-4.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,5.0,BMO,"5,000,000",NaN,RBC,"5,000,000",0,0,"1,535","2,359",29260ZAL0,44810ZCF5
5,ENBCN 4 11/15/49,RCICN 3.8 03/15/32,-5.3,4.6,-3.5,-50.0,-1.8,44.7,99.4,Pipeline,FIXED,ENBCN,209.2,24.3,USD,ENB CN Equity,5.7,995,BBB+,5-7,10.1-25.1,Cable/Telco,FIXED,RCICN,148.6,6.7,USD,RCI/B CN Equity,3.3,534,BBB-,3-5,5.1-7.1,NaN,NaN,,NaN,NaN,,0,0,0,0,29250NBA2,775109CC3
6,DE 2.81 01/19/29,HOMEQU 6.552 10/18/27,-115.2,4.5,-115.2,-156.0,0.0,40.8,99.8,Industrials,FIXED,DE,42.1,3.5,CAD,DE US Equity,3.5,325,A,3-5,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,10.0,TD,"2,000,000",NaN,CIBC,"3,000,000",0,1,651,658,47788ZAM3,43739CAB7
7,MPTFIN 3.457 11/30/29,RCICN 3.8 03/15/32,78.6,4.5,78.8,16.3,-0.2,62.3,99.4,Other,FIXED,MPTFIN,205.6,3.8,CAD,1688820D CN Equity,5.6,333,NR,5-7,4.1-5.1,Cable/Telco,FIXED,RCICN,148.6,6.7,USD,RCI/B CN Equity,3.3,534,BBB-,3-5,5.1-7.1,NaN,NaN,,NaN,NaN,,0,0,0,0,55342VAB8,775109CC3
8,FTSCN 4.198 04/27/52,RCICN 3.8 03/15/32,53.8,4.4,53.8,0.7,0.0,53.1,99.8,Utility,FIXED,FTSCN,179.3,23.0,CAD,FTS CN Equity,3.2,"1,143",NR,3-5,>25.1,Cable/Telco,FIXED,RCICN,148.6,6.7,USD,RCI/B CN Equity,3.3,534,BBB-,3-5,5.1-7.1,NaN,NaN,,NaN,NaN,,0,0,0,0,65135PAG0,775109CC3
9,ATH 4.86 08/27/26,HOMEQU 6.552 10/18/27,-109.7,4.4,-109.7,-154.2,0.0,44.5,99.8,Financial Maples,FIXED,ATH,65.3,1.1,USD,ATH US Equity,0.9,108,A+,0.50-1,1-2.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,NaN,NaN,,NaN,CIBC,"3,000,000",0,1,0,658,04685A4A6,43739CAB7


In [6]:
# All CAD Bonds IN the Universe

# Create z_cad_universe DataFrame - only CAD currency pairs
z_cad_universe = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Yrs (Cvn)_1'] > 1) & 
    (z_all_enhanced['Yrs (Cvn)_2'] > 1)
].copy()

display(format_for_display(z_cad_universe.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
0,HOMEQU 5.82 08/13/29,HOMEQU 6.552 10/18/27,51.0,5.2,51.0,26.0,0.0,25.1,99.8,Mortgage Provider,FIXED,HOMEQU,212.0,4.0,CAD,HEQ CN Equity,0.9,363,NR,0.50-1,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,3.0,RBC,"3,000,000",NaN,CIBC,"3,000,000",0,1,"1,089",658,43739CAC5,43739CAB7
4,ENBGAS 5.46 10/06/28,HYDONE 1.69 01/16/31,23.4,4.6,23.4,3.3,0.0,20.1,99.8,Utility,FIXED,ENBGAS,48.8,3.1,CAD,ENB CN Equity,1.8,307,A-,1-2,3.1-4.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,5.0,BMO,"5,000,000",NaN,RBC,"5,000,000",0,0,"1,535","2,359",29260ZAL0,44810ZCF5
6,DE 2.81 01/19/29,HOMEQU 6.552 10/18/27,-115.2,4.5,-115.2,-156.0,0.0,40.8,99.8,Industrials,FIXED,DE,42.1,3.5,CAD,DE US Equity,3.5,325,A,3-5,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,10.0,TD,"2,000,000",NaN,CIBC,"3,000,000",0,1,651,658,47788ZAM3,43739CAB7
10,CP 2.54 02/28/28,HYDONE 1.69 01/16/31,12.9,4.4,12.9,-3.8,0.0,16.7,99.8,Infastructure,FIXED,CP,39.2,2.6,CAD,CP CN Equity,3.6,247,BBB+,3-5,2.1-3.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,5.0,NBF,"3,000,000",NaN,RBC,"5,000,000",0,0,741,"2,359",13645RBC7,44810ZCF5
11,CAT 4.51 03/05/27,GWOCN 3.337 02/28/28,11.2,4.4,11.2,-7.7,0.0,19.0,99.8,Industrials,FIXED,CAT,44.0,1.6,CAD,CAT US Equity,1.4,161,A,1-2,1-2.1,Insurance Senior,FIXED,GWOCN,45.3,2.4,CAD,POW CN Equity,7.4,227,A,>7,2.1-3.1,5.0,TD,"3,000,000",4.0,TD,"5,000,000",0,0,483,"1,134",14913LAC4,39138CAH9
13,CTLONE 5.877 11/10/26,HOMEQU 6.552 10/18/27,-91.6,4.3,-91.6,-137.7,0.0,46.0,99.8,Non Major Sub Debt,FIXED,CTLONE,69.5,1.3,CAD,1568Z CN Equity,1.7,131,A,1-2,1-2.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,NaN,None,,NaN,CIBC,"3,000,000",1,1,0,658,154728AV1,43739CAB7
14,DE 1.34 09/08/27,HOMEQU 6.552 10/18/27,-139.0,4.3,-139.0,-184.8,0.0,45.9,99.8,Industrials,FIXED,DE,20.8,2.1,CAD,DE US Equity,4.8,203,A,3-5,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,NaN,None,,NaN,CIBC,"3,000,000",0,1,0,658,47788ZAH4,43739CAB7
15,GTAAIR 3.26 06/01/37,HYDONE 1.69 01/16/31,76.1,4.3,76.1,53.5,0.0,22.6,99.8,Airport,FIXED,GTAAIR,97.2,11.9,CAD,1318Z CN Equity,7.3,844,A+,>7,10.1-25.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,5.0,BMO,"2,000,000",NaN,RBC,"5,000,000",0,0,"1,688","2,359",39191ZBB4,44810ZCF5
18,GTAAIR 2 ¾ 10/17/39,HYDONE 1.69 01/16/31,75.4,4.3,75.4,56.6,0.0,18.8,99.8,Airport,FIXED,GTAAIR,100.8,14.3,CAD,1318Z CN Equity,5.7,905,A+,5-7,10.1-25.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,5.0,SCM,"3,000,000",NaN,RBC,"5,000,000",0,0,"2,715","2,359",39191ZBD0,44810ZCF5
19,HICITR 4.065 02/22/59,HOMEQU 6.552 10/18/27,38.1,4.3,38.1,-9.3,0.0,47.4,99.8,Other,FIXED,HICITR,200.8,20.6,CAD,1690248D CN Equity,6.4,"1,056",NR,5-7,>25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,NaN,NaN,,NaN,CIBC,"3,000,000",0,1,0,658,42834JAB8,43739CAB7


In [7]:
# Comparing Same Term

z_term = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Yrs (Mat) Bucket_1'] == z_all_enhanced['Yrs (Mat) Bucket_2'])
].copy()

display(format_for_display(z_term.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
14,DE 1.34 09/08/27,HOMEQU 6.552 10/18/27,-139.0,4.3,-139.0,-184.8,0.0,45.9,99.8,Industrials,FIXED,DE,20.8,2.1,CAD,DE US Equity,4.8,203,A,3-5,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,NaN,None,,NaN,CIBC,"3,000,000",0,1,0,658,47788ZAH4,43739CAB7
20,CM 4.9 06/12/34,RY 1.67 01/28/33,65.7,4.3,65.7,47.6,0.0,18.1,99.8,NVCC Sub Debt,VARIABLE,CM,107.8,3.9,CAD,CM CN Equity,1.1,365,BBB+,1-2,7.1-10.1,NVCC Sub Debt,VARIABLE,RY,50.8,2.5,CAD,RY CN Equity,4.5,235,A-,3-5,7.1-10.1,4.0,SCM,"5,000,000",NaN,TD,"5,000,000",0,0,"1,824","1,177",13607PBA1,780086SY1
25,CNQCN 2 ½ 01/17/28,HOMEQU 6.552 10/18/27,-97.4,4.2,-97.4,-144.7,0.0,47.2,99.8,Oil & Gas,FIXED,CNQCN,62.5,2.5,CAD,CNQ CN Equity,4.7,235,BBB,3-5,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,SCM,"3,000,000",NaN,CIBC,"3,000,000",0,1,705,658,13638ZDX0,43739CAB7
32,DE 4.38 07/11/28,HOMEQU 6.552 10/18/27,-105.5,4.1,-105.5,-152.5,0.0,47.1,99.8,Industrials,FIXED,DE,55.8,3.0,CAD,DE US Equity,1.0,287,A,1-2,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,4.0,TD,"5,000,000",NaN,CIBC,"3,000,000",0,1,"1,434",658,47788ZAS0,43739CAB7
34,DOLCN 1.505 09/20/27,HOMEQU 6.552 10/18/27,-127.6,4.1,-127.6,-173.3,0.0,45.8,99.8,Retail/Grocers,FIXED,DOLCN,30.3,2.2,CAD,DOL CN Equity,4.8,205,NR,3-5,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,3.0,BMO,"3,000,000",NaN,CIBC,"3,000,000",0,1,616,658,25675TAK3,43739CAB7
38,BMO 3.19 03/01/28,HOMEQU 6.552 10/18/27,-115.8,4.1,-115.8,-172.8,0.0,57.0,99.8,Dep Note,FIXED,BMO,43.8,2.6,CAD,BMO CN Equity,7.4,250,AA,>7,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,4.0,SCM,"10,000,000",NaN,CIBC,"3,000,000",0,1,"2,502",658,06368BTX6,43739CAB7
60,BNS 3.1 02/02/28,HOMEQU 6.552 10/18/27,-120.0,4.0,-120.0,-173.5,0.0,53.5,99.8,Dep Note,FIXED,BNS,40.0,2.6,CAD,BNS CN Equity,7.4,243,AA-,>7,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,3.0,BMO,"5,000,000",NaN,CIBC,"3,000,000",0,1,"1,214",658,064151S95,43739CAB7
66,BAC 3.615 03/16/28,HOMEQU 6.552 10/18/27,-98.7,4.0,-98.7,-143.8,0.0,45.2,99.8,Financial Maples,VARIABLE,BAC,63.8,1.7,CAD,BAC US Equity,3.3,162,A,3-5,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,3.0,NBF,"10,000,000",NaN,CIBC,"3,000,000",0,1,"1,618",658,060505GF5,43739CAB7
73,FFHCN 4 ¼ 12/06/27,HOMEQU 6.552 10/18/27,-94.5,4.0,-94.5,-143.1,0.0,48.6,99.8,Insurance Senior,FIXED,FFHCN,63.4,2.1,CAD,FFH CN Equity,7.6,207,BBB+,>7,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,TD,"5,000,000",NaN,CIBC,"3,000,000",0,1,"1,037",658,303901AZ5,43739CAB7
87,BMO 3.19 03/01/28,GWOCN 3.337 02/28/28,10.8,3.9,10.8,-15.2,0.0,26.1,99.8,Dep Note,FIXED,BMO,43.8,2.6,CAD,BMO CN Equity,7.4,250,AA,>7,2.1-3.1,Insurance Senior,FIXED,GWOCN,45.3,2.4,CAD,POW CN Equity,7.4,227,A,>7,2.1-3.1,4.0,SCM,"10,000,000",4.0,TD,"5,000,000",0,0,"2,502","1,134",06368BTX6,39138CAH9


In [8]:
# Comparing Same Sector

z_sector = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Custom_Sector_1'] == z_all_enhanced['Custom_Sector_2'])
].copy()

display(format_for_display(z_sector.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
0,HOMEQU 5.82 08/13/29,HOMEQU 6.552 10/18/27,51.0,5.2,51.0,26.0,0.0,25.1,99.8,Mortgage Provider,FIXED,HOMEQU,212.0,4.0,CAD,HEQ CN Equity,0.9,363,NR,0.50-1,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,3.0,RBC,"3,000,000",NaN,CIBC,"3,000,000",0,1,"1,089",658,43739CAC5,43739CAB7
4,ENBGAS 5.46 10/06/28,HYDONE 1.69 01/16/31,23.4,4.6,23.4,3.3,0.0,20.1,99.8,Utility,FIXED,ENBGAS,48.8,3.1,CAD,ENB CN Equity,1.8,307,A-,1-2,3.1-4.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,5.0,BMO,"5,000,000",NaN,RBC,"5,000,000",0,0,"1,535","2,359",29260ZAL0,44810ZCF5
20,CM 4.9 06/12/34,RY 1.67 01/28/33,65.7,4.3,65.7,47.6,0.0,18.1,99.8,NVCC Sub Debt,VARIABLE,CM,107.8,3.9,CAD,CM CN Equity,1.1,365,BBB+,1-2,7.1-10.1,NVCC Sub Debt,VARIABLE,RY,50.8,2.5,CAD,RY CN Equity,4.5,235,A-,3-5,7.1-10.1,4.0,SCM,"5,000,000",NaN,TD,"5,000,000",0,0,"1,824","1,177",13607PBA1,780086SY1
24,ENBGAS 4 08/22/44,HYDONE 1.69 01/16/31,98.6,4.2,98.6,80.0,0.0,18.5,99.8,Utility,FIXED,ENBGAS,125.3,19.1,CAD,ENB CN Equity,10.9,"1,122",A-,>7,10.1-25.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,3.0,TD,"2,000,000",NaN,RBC,"5,000,000",0,0,"2,243","2,359",29290ZAP5,44810ZCF5
31,HYDONE 3.79 07/31/62,LOWMAT 2.433 05/14/31,76.5,4.1,79.8,53.8,-3.3,22.7,99.0,Utility,FIXED,HYDONE,115.4,37.0,CAD,H CN Equity,13.0,"1,445",A,>7,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,RBC,"5,000,000",10.0,TD,"3,000,000",0,0,"7,226","1,503",44810ZBG4,548243AJ3
36,ALTALK 4.09 06/30/45,HYDONE 1.69 01/16/31,88.4,4.1,88.4,69.5,0.0,18.9,99.8,Utility,FIXED,ALTALK,114.4,20.0,CAD,BRK/A US Equity,10.0,"1,179",A-,>7,10.1-25.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,1.0,NBF,"5,000,000",NaN,RBC,"5,000,000",0,0,"5,894","2,359",02135ZAQ9,44810ZCF5
37,ENBGAS 3.8 06/01/46,HYDONE 1.69 01/16/31,96.0,4.1,96.0,78.1,0.0,17.9,99.8,Utility,FIXED,ENBGAS,122.9,20.9,CAD,ENB CN Equity,9.1,"1,151",A-,>7,10.1-25.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,3.0,TD,"2,000,000",NaN,RBC,"5,000,000",0,0,"2,303","2,359",90664ZAW0,44810ZCF5
39,ACICN 4.26 12/05/28,HYDONE 1.69 01/16/31,53.8,4.1,53.8,33.4,0.0,20.4,99.8,Utility,FIXED,ACICN,79.4,3.1,CAD,ACI CN Equity,6.6,298,BBB+,5-7,3.1-4.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,5.0,TD,"3,000,000",NaN,RBC,"5,000,000",0,0,894,"2,359",02091ZAA6,44810ZCF5
42,ENBGAS 4.2 06/02/44,HYDONE 1.69 01/16/31,98.2,4.1,98.2,80.9,0.0,17.3,99.8,Utility,FIXED,ENBGAS,125.5,18.9,CAD,ENB CN Equity,11.1,"1,134",A-,>7,10.1-25.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,3.0,TD,"2,000,000",NaN,RBC,"5,000,000",0,0,"2,268","2,359",90664ZAD2,44810ZCF5
43,FTSCN 3 ⅜ 04/13/45,HYDONE 1.69 01/16/31,91.7,4.1,91.7,71.1,0.0,20.5,99.8,Utility,FIXED,FTSCN,118.8,19.7,CAD,FTS CN Equity,10.3,"1,090",A-,>7,10.1-25.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,5.0,TD,"2,000,000",NaN,RBC,"5,000,000",0,0,"2,180","2,359",34959ZAB8,44810ZCF5


In [9]:
# Comparing Same Ticker

z_ticker = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Ticker_1'] == z_all_enhanced['Ticker_2'])
].copy()

display(format_for_display(z_ticker.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
0,HOMEQU 5.82 08/13/29,HOMEQU 6.552 10/18/27,51.0,5.2,51.0,26.0,0.0,25.1,99.8,Mortgage Provider,FIXED,HOMEQU,212.0,4.0,CAD,HEQ CN Equity,0.9,363,NR,0.50-1,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,3.0,RBC,"3,000,000",NaN,CIBC,"3,000,000",0,1,"1,089",658,43739CAC5,43739CAB7
157,RY 4.642 01/17/28,RY 5.228 06/24/30,-7.5,3.8,-7.5,-20.2,0.0,12.7,99.8,Bail In,FIXED,RY,52.3,2.5,CAD,RY CN Equity,2.5,243,A+,2-3,2.1-3.1,Bail In,FIXED,RY,61.9,4.9,CAD,RY CN Equity,2.1,466,A+,2-3,4.1-5.1,3.0,BMO,"5,000,000",5.0,TD,"10,000,000",1,0,"1,216","4,657",780086WG5,780086ZH0
209,RCICN 3.65 03/31/27,RCICN 5.9 09/21/33,-49.6,3.8,-49.6,-67.2,0.0,17.6,99.8,Cable/Telco,FIXED,RCICN,70.6,1.5,CAD,RCI/B CN Equity,5.3,150,BBB-,5-7,1-2.1,Cable/Telco,FIXED,RCICN,123.4,7.9,CAD,RCI/B CN Equity,1.8,692,BBB-,1-2,7.1-10.1,4.0,NBF,"10,000,000",4.0,RBC,"10,000,000",0,0,"1,499","6,921",775109BK6,775109DA6
221,BNS 2.62 12/02/26,BNS 4.442 11/15/35,-70.7,3.8,-70.7,-91.3,0.0,20.6,99.8,Dep Note,FIXED,BNS,21.8,1.4,CAD,BNS CN Equity,8.6,134,AA-,>7,1-2.1,Bail In,VARIABLE,BNS,98.9,9.3,CAD,BNS CN Equity,0.7,756,A,0.50-1,10.1-25.1,5.0,RBC,"10,000,000",4.0,TD,"10,000,000",0,0,"1,342","7,560",064151QE6,06418MZ49
739,RCICN 3.65 03/31/27,RCICN 5.8 09/21/30,-22.9,3.5,-22.9,-41.2,0.0,18.3,99.8,Cable/Telco,FIXED,RCICN,70.6,1.5,CAD,RCI/B CN Equity,5.3,150,BBB-,5-7,1-2.1,Cable/Telco,FIXED,RCICN,95.9,5.0,CAD,RCI/B CN Equity,1.8,471,BBB-,1-2,5.1-7.1,4.0,NBF,"10,000,000",5.0,BMO,"5,000,000",0,0,"1,499","2,355",775109BK6,775109CW9
816,CHPUCN 2.456 11/30/26,CHPUCN 2.981 03/04/30,-38.2,3.4,-38.2,-60.7,0.0,22.5,99.8,REIT,FIXED,CHPUCN,43.6,1.4,CAD,CHP-U CN Equity,3.6,133,BBB+,3-5,1-2.1,REIT,FIXED,CHPUCN,95.0,4.6,CAD,CHP-U CN Equity,5.4,409,BBB+,5-7,4.1-5.1,5.0,RBC,"5,000,000",NaN,CIBC,"5,000,000",0,0,663,"2,046",17039AAV8,17039AAS5
1001,CHPUCN 2.848 05/21/27,CHPUCN 2.981 03/04/30,-28.5,3.4,-28.5,-48.2,0.0,19.7,99.8,REIT,FIXED,CHPUCN,54.4,1.8,CAD,CHP-U CN Equity,5.1,177,BBB+,5-7,1-2.1,REIT,FIXED,CHPUCN,95.0,4.6,CAD,CHP-U CN Equity,5.4,409,BBB+,5-7,4.1-5.1,NaN,None,,NaN,CIBC,"5,000,000",0,0,0,"2,046",17039AAU0,17039AAS5
1572,BNS 2.62 12/02/26,BNS 4.68 02/01/29,-37.9,3.2,-37.9,-58.1,0.0,20.2,99.8,Dep Note,FIXED,BNS,21.8,1.4,CAD,BNS CN Equity,8.6,134,AA-,>7,1-2.1,Bail In,FIXED,BNS,63.9,3.6,CAD,BNS CN Equity,1.4,338,A,1-2,3.1-4.1,5.0,RBC,"10,000,000",3.0,BMO,"20,000,000",0,1,"1,342","6,756",064151QE6,064164QM1
1706,HYDONE 1.41 10/15/27,HYDONE 1.69 01/16/31,-20.0,3.2,-20.0,-43.0,0.0,23.0,99.8,Other,FIXED,HYDONE,7.3,2.3,CAD,H CN Equity,4.7,212,A-,3-5,2.1-3.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,NaN,None,,NaN,RBC,"5,000,000",0,0,0,"2,359",448811AC3,44810ZCF5
1708,TORHYD 3.27 10/18/51,TORHYD 3.96 04/09/63,-3.0,3.2,-3.0,-13.0,0.0,10.0,99.8,Utility,FIXED,TORHYD,104.6,26.3,CAD,3806Z CN Equity,3.7,"1,223",A,3-5,>25.1,Utility,FIXED,TORHYD,113.0,37.7,CAD,3806Z CN Equity,12.3,"1,487",A,>7,>25.1,NaN,None,,NaN,RBC,"2,000,000",0,0,0,"2,974",89119ZAQ6,89119ZAF0


In [10]:
# Where CAD Bonds May Appear Cheap vs USD Bonds


z_cad_vs_usd = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'USD') &
    (z_all_enhanced['Custom_Sector_1'] == z_all_enhanced['Custom_Sector_2']) &
    (z_all_enhanced['Yrs (Mat) Bucket_1'] == z_all_enhanced['Yrs (Mat) Bucket_2']) &
    (z_all_enhanced['Ticker_1'] == z_all_enhanced['Ticker_2'])
].copy()

display(format_for_display(z_cad_vs_usd.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
121828,WFC 2.493 02/18/27,WFC 5.254 12/11/26,38.9,1.5,41.2,11.4,-2.3,27.5,97.8,Financial Maples,FIXED,WFC,52.2,1.6,CAD,WFC US Equity,5.4,153,A,5-7,1-2.1,Financial Maples,FIXED,WFC,49.1,1.3,USD,WFC US Equity,1.6,128,AA-,1-2,1-2.1,4.0,TD,"5,000,000",NaN,NaN,,0,0,765,0,949746SZ5,94988J6F9
308234,ENBCN 5 ⅜ 09/27/2077,ENBCN 7 ⅜ 01/15/2083,-13.7,0.7,26.7,-128.3,-40.3,114.7,73.6,Non Financial Hybrids,VARIABLE,ENBCN,208.2,2.2,CAD,ENB CN Equity,7.8,208,BB+,>7,>25.1,Non Financial Hybrids,VARIABLE,ENBCN,212.1,2.3,USD,ENB CN Equity,2.8,213,BB+,2-3,>25.1,10.0,BMO,"5,000,000",NaN,NaN,,0,0,"1,040",0,29250NAT2,29250NBN4
312290,BNS 3.7 07/27/2081,BNS 8 ⅝ 10/27/2082,962.1,0.7,1229.8,464.6,-267.6,497.5,67.3,Financial Hybrid,VARIABLE,BNS,1158.5,1.0,CAD,BNS CN Equity,4.1,89,BBB,3-5,>25.1,Financial Hybrid,VARIABLE,BNS,194.2,2.3,USD,BNS CN Equity,2.7,221,BBB-,2-3,>25.1,44.0,RBC,"3,000,000",NaN,NaN,,1,0,266,0,06415FRB0,0641598S8
312844,ENBCN 6 ⅝ 04/12/2078,ENBCN 7 ⅜ 01/15/2083,-21.2,0.7,20.1,-151.0,-41.3,129.8,74.4,Non Financial Hybrids,VARIABLE,ENBCN,198.6,2.7,CAD,ENB CN Equity,7.3,261,BB+,>7,>25.1,Non Financial Hybrids,VARIABLE,ENBCN,212.1,2.3,USD,ENB CN Equity,2.8,213,BB+,2-3,>25.1,10.0,RBC,"2,000,000",NaN,NaN,,0,0,523,0,29250NAX3,29250NBN4
316457,ENBCN 5.32 08/22/54,ENBCN 8 ¼ 01/15/2084,-54.1,0.7,-2.4,-178.3,-51.8,124.2,77.2,Pipeline,FIXED,ENBCN,157.2,29.1,CAD,ENB CN Equity,0.9,"1,442",BBB+,0.50-1,>25.1,Pipeline,VARIABLE,ENBCN,248.8,3.3,USD,ENB CN Equity,1.8,303,BB+,1-2,>25.1,5.0,RBC,"2,000,000",NaN,NaN,,0,0,"2,884",0,29251ZCE5,29250NBS3
317488,ENBCN 6.51 11/09/52,ENBCN 8 ¼ 01/15/2084,-54.9,0.7,-3.2,-178.1,-51.7,123.2,76.4,Pipeline,FIXED,ENBCN,156.1,26.8,CAD,ENB CN Equity,2.7,"1,564",BBB+,2-3,>25.1,Pipeline,VARIABLE,ENBCN,248.8,3.3,USD,ENB CN Equity,1.8,303,BB+,1-2,>25.1,5.0,RBC,"2,000,000",NaN,NaN,,0,0,"3,128",0,29251ZBX4,29250NBS3
320047,ENBCN 4.1 09/21/51,ENBCN 8 ¼ 01/15/2084,-55.6,0.7,-2.1,-179.5,-53.5,123.9,74.8,Pipeline,FIXED,ENBCN,155.6,26.2,CAD,ENB CN Equity,3.8,"1,207",BBB+,3-5,>25.1,Pipeline,VARIABLE,ENBCN,248.8,3.3,USD,ENB CN Equity,1.8,303,BB+,1-2,>25.1,5.0,RBC,"2,000,000",NaN,NaN,,0,0,"2,414",0,29251ZBV8,29250NBS3
321212,ENBCN 5.76 05/26/53,ENBCN 8 ¼ 01/15/2084,-55.0,0.7,-2.1,-179.5,-52.8,124.6,76.4,Pipeline,FIXED,ENBCN,156.5,27.4,CAD,ENB CN Equity,2.1,"1,470",BBB+,2-3,>25.1,Pipeline,VARIABLE,ENBCN,248.8,3.3,USD,ENB CN Equity,1.8,303,BB+,1-2,>25.1,5.0,RBC,"2,000,000",NaN,NaN,,0,0,"2,941",0,29251ZCB1,29250NBS3
329457,BNS 3.7 07/27/2081,BNS 8 01/27/2084,923.5,0.7,1217.9,433.0,-294.3,490.6,67.3,Financial Hybrid,VARIABLE,BNS,1158.5,1.0,CAD,BNS CN Equity,4.1,89,BBB,3-5,>25.1,Financial Hybrid,VARIABLE,BNS,240.7,3.5,USD,BNS CN Equity,1.5,329,BBB-,1-2,>25.1,44.0,RBC,"3,000,000",NaN,NaN,,1,0,266,0,06415FRB0,0641598V1
359920,ENBCN 5 ⅜ 09/27/2077,ENBCN 6 ¼ 03/01/2078,-32.5,0.6,-10.1,-107.0,-22.5,74.5,70.4,Non Financial Hybrids,VARIABLE,ENBCN,208.2,2.2,CAD,ENB CN Equity,7.8,208,BB+,>7,>25.1,Non Financial Hybrids,VARIABLE,ENBCN,254.0,2.6,USD,ENB CN Equity,7.4,240,BB+,>7,>25.1,10.0,BMO,"5,000,000",NaN,NaN,,0,0,"1,040",0,29250NAT2,29250NAW5


In [11]:
# Where CAD Bonds May Appear Rich vs USD Bonds


z_usd_vs_cad = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'USD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Custom_Sector_1'] == z_all_enhanced['Custom_Sector_2']) &
    (z_all_enhanced['Yrs (Mat) Bucket_1'] == z_all_enhanced['Yrs (Mat) Bucket_2']) &
    (z_all_enhanced['Ticker_1'] == z_all_enhanced['Ticker_2'])
].copy()

display(format_for_display(z_usd_vs_cad.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
19660,PLD 1 ¼ 10/15/30,PLD 5 ¼ 01/15/31,-28.4,2.4,-28.4,-67.7,0.0,39.3,99.8,REIT,FIXED,PLD,92.6,5.3,USD,PLD US Equity,4.9,422,A,3-5,5.1-7.1,REIT,FIXED,PLD,84.6,5.3,CAD,PLD US Equity,2.7,486,A,2-3,5.1-7.1,NaN,NaN,,5.0,NBF,"5,000,000",0,1,0,"2,431",74340XBR1,74340XCD1
36024,BCECN 3.65 03/17/51,BCECN 5.6 08/11/53,-42.6,2.1,-39.3,-83.7,-3.4,41.1,99.0,Cable/Telco,FIXED,BCECN,189.6,25.7,USD,BCE CN Equity,4.3,"1,007",BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.7,27.6,CAD,BCE CN Equity,1.9,"1,458",BBB,1-2,>25.1,NaN,NaN,,4.0,TD,"5,000,000",0,0,0,"7,289",0778FPAF6,07813ZCP7
36696,BCECN 3.65 03/17/51,BCECN 5.15 02/09/53,-42.2,2.1,-38.8,-85.6,-3.4,43.4,99.4,Cable/Telco,FIXED,BCECN,189.6,25.7,USD,BCE CN Equity,4.3,"1,007",BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.6,27.6,CAD,BCE CN Equity,2.4,"1,395",BBB,2-3,>25.1,NaN,NaN,,3.0,TD,"5,000,000",0,0,0,"6,973",0778FPAF6,07813ZCM4
39005,ALACN 7.2 10/15/54,ALACN 8.9 11/10/2083,62.9,2.1,68.2,-35.4,-5.3,98.3,96.2,Non Financial Hybrids,VARIABLE,ALACN,332.1,9.0,USD,ALA CN Equity,0.8,660,BB,0.50-1,>25.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,NaN,NaN,,16.0,CIBC,"2,000,000",0,0,0,649,021361AD2,021361AC4
40095,TOYOTA 2.15 02/13/30,TOYOTA 3.73 10/02/29,-10.0,2.1,-9.5,-28.1,-0.5,18.0,98.2,Auto,FIXED,TOYOTA,96.0,4.6,USD,7203 JP Equity,5.4,386,A+,5-7,4.1-5.1,Auto,FIXED,TOYOTA,68.8,4.2,CAD,7203 JP Equity,0.8,388,A+,0.50-1,4.1-5.1,NaN,NaN,,5.0,CIBC,"5,000,000",0,0,0,"1,939",89236TGU3,892329CA9
42762,PLD 1 ⅝ 03/15/31,PLD 5 ¼ 01/15/31,-14.8,2.1,-14.6,-51.9,-0.2,37.1,99.4,REIT,FIXED,PLD,110.7,5.7,USD,PLD US Equity,4.4,449,A,3-5,5.1-7.1,REIT,FIXED,PLD,84.6,5.3,CAD,PLD US Equity,2.7,486,A,2-3,5.1-7.1,NaN,NaN,,5.0,NBF,"5,000,000",0,1,0,"2,431",74340XBS9,74340XCD1
43238,BCECN 3.65 08/15/52,BCECN 5.15 02/09/53,-35.0,2.0,-32.8,-84.4,-2.2,49.4,99.4,Cable/Telco,FIXED,BCECN,201.7,27.1,USD,BCE CN Equity,3.4,995,BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.6,27.6,CAD,BCE CN Equity,2.4,"1,395",BBB,2-3,>25.1,NaN,NaN,,3.0,TD,"5,000,000",0,0,0,"6,973",0778FPAJ8,07813ZCM4
43287,BCECN 3.65 08/15/52,BCECN 5.6 08/11/53,-35.5,2.0,-33.2,-83.5,-2.2,48.0,99.4,Cable/Telco,FIXED,BCECN,201.7,27.1,USD,BCE CN Equity,3.4,995,BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.7,27.6,CAD,BCE CN Equity,1.9,"1,458",BBB,1-2,>25.1,NaN,NaN,,4.0,TD,"5,000,000",0,0,0,"7,289",0778FPAJ8,07813ZCP7
46675,RY 1.4 11/02/26,RY 2.328 01/28/27,-7.0,2.0,-7.0,-33.2,0.0,26.3,99.8,Bail In,FIXED,RY,46.7,1.3,USD,RY CN Equity,3.8,122,A+,3-5,1-2.1,Bail In,FIXED,RY,32.9,1.5,CAD,RY CN Equity,5.5,148,A+,5-7,1-2.1,NaN,NaN,,5.0,NBF,"20,000,000",0,0,0,"2,957",78016EZZ3,780086RQ9
52897,BCECN 4.3 07/29/49,BCECN 4.35 12/18/45,-30.6,2.0,-29.2,-84.3,-1.4,53.7,99.4,Cable/Telco,FIXED,BCECN,207.4,24.0,USD,BCE CN Equity,6.2,"1,030",BBB,5-7,10.1-25.1,Cable/Telco,FIXED,BCECN,162.2,20.4,CAD,BCE CN Equity,10.3,"1,124",BBB,>7,10.1-25.1,NaN,NaN,,5.0,TD,"3,000,000",0,0,0,"3,372",0778FPAB5,07813ZBR4


In [12]:
# Which Bonds in Our Portfolio May Look Rich vs Universe

z_port_vs_universe = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
0,HOMEQU 5.82 08/13/29,HOMEQU 6.552 10/18/27,51.0,5.2,51.0,26.0,0.0,25.1,99.8,Mortgage Provider,FIXED,HOMEQU,212.0,4.0,CAD,HEQ CN Equity,0.9,363,NR,0.50-1,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,3.0,RBC,"3,000,000",NaN,CIBC,"3,000,000",0,1,"1,089",658,43739CAC5,43739CAB7
6,DE 2.81 01/19/29,HOMEQU 6.552 10/18/27,-115.2,4.5,-115.2,-156.0,0.0,40.8,99.8,Industrials,FIXED,DE,42.1,3.5,CAD,DE US Equity,3.5,325,A,3-5,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,10.0,TD,"2,000,000",NaN,CIBC,"3,000,000",0,1,651,658,47788ZAM3,43739CAB7
13,CTLONE 5.877 11/10/26,HOMEQU 6.552 10/18/27,-91.6,4.3,-91.6,-137.7,0.0,46.0,99.8,Non Major Sub Debt,FIXED,CTLONE,69.5,1.3,CAD,1568Z CN Equity,1.7,131,A,1-2,1-2.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,NaN,None,,NaN,CIBC,"3,000,000",1,1,0,658,154728AV1,43739CAB7
14,DE 1.34 09/08/27,HOMEQU 6.552 10/18/27,-139.0,4.3,-139.0,-184.8,0.0,45.9,99.8,Industrials,FIXED,DE,20.8,2.1,CAD,DE US Equity,4.8,203,A,3-5,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,NaN,None,,NaN,CIBC,"3,000,000",0,1,0,658,47788ZAH4,43739CAB7
19,HICITR 4.065 02/22/59,HOMEQU 6.552 10/18/27,38.1,4.3,38.1,-9.3,0.0,47.4,99.8,Other,FIXED,HICITR,200.8,20.6,CAD,1690248D CN Equity,6.4,"1,056",NR,5-7,>25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,NaN,NaN,,NaN,CIBC,"3,000,000",0,1,0,658,42834JAB8,43739CAB7
21,HICITR 3.91 02/22/49,HOMEQU 6.552 10/18/27,62.0,4.3,62.0,11.7,0.0,50.3,99.8,Other,FIXED,HICITR,222.5,13.7,CAD,1690248D CN Equity,6.4,841,NR,5-7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,NaN,NaN,,NaN,CIBC,"3,000,000",0,1,0,658,42834JAA0,43739CAB7
25,CNQCN 2 ½ 01/17/28,HOMEQU 6.552 10/18/27,-97.4,4.2,-97.4,-144.7,0.0,47.2,99.8,Oil & Gas,FIXED,CNQCN,62.5,2.5,CAD,CNQ CN Equity,4.7,235,BBB,3-5,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,SCM,"3,000,000",NaN,CIBC,"3,000,000",0,1,705,658,13638ZDX0,43739CAB7
26,DE 4.95 06/14/27,HOMEQU 6.552 10/18/27,-115.8,4.2,-115.8,-164.1,0.0,48.3,99.8,Industrials,FIXED,DE,44.5,1.9,CAD,DE US Equity,2.1,188,A,2-3,1-2.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,NBF,"5,000,000",NaN,CIBC,"3,000,000",0,1,942,658,47788ZAN1,43739CAB7
27,ACICN 4.26 12/05/28,HOMEQU 6.552 10/18/27,-80.2,4.2,-80.2,-131.2,0.0,51.0,99.8,Utility,FIXED,ACICN,79.4,3.1,CAD,ACI CN Equity,6.6,298,BBB+,5-7,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,TD,"3,000,000",NaN,CIBC,"3,000,000",0,1,894,658,02091ZAA6,43739CAB7
32,DE 4.38 07/11/28,HOMEQU 6.552 10/18/27,-105.5,4.1,-105.5,-152.5,0.0,47.1,99.8,Industrials,FIXED,DE,55.8,3.0,CAD,DE US Equity,1.0,287,A,1-2,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,4.0,TD,"5,000,000",NaN,CIBC,"3,000,000",0,1,"1,434",658,47788ZAS0,43739CAB7


In [13]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute

z_port_vs_universe_2way_liquidity = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['CUSIP_2 CR01 @ Best Bid'] >2000) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_2way_liquidity.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
1628,ENBCN 5.36 05/26/33,PPLCN 3.53 12/10/31,13.3,3.2,15.2,-1.4,-1.9,14.6,99.4,Pipeline,FIXED,ENBCN,115.5,7.6,CAD,ENB CN Equity,2.1,660,BBB+,2-3,7.1-10.1,Midstream,FIXED,PPLCN,107.7,6.4,CAD,PPL CN Equity,3.6,541,BBB,3-5,5.1-7.1,5.0,RBC,"5,000,000",5.0,RBC,"5,000,000",0,1,"3,301","2,707",29251ZBZ9,70632ZAS0
1671,GTAAIR 5.63 06/07/40,WSPCN 4.754 09/12/34,-0.8,3.2,3.5,-24.8,-4.3,24.0,99.4,Airport,FIXED,GTAAIR,116.4,14.9,CAD,1318Z CN Equity,15.1,"1,109",A+,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,5.0,NBF,"2,000,000",NaN,NBF,"5,000,000",0,1,"2,218","3,625",39191ZAW9,92938WAD5
1694,ENBGAS 5.2 07/23/40,WSPCN 4.754 09/12/34,13.0,3.2,18.0,-7.6,-5.0,20.6,99.4,Utility,FIXED,ENBGAS,130.9,15.0,CAD,ENB CN Equity,15.0,"1,063",A-,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,4.0,BMO,"2,000,000",NaN,NBF,"5,000,000",0,1,"2,126","3,625",90664ZAT7,92938WAD5
1845,GTAAIR 5.3 02/25/41,WSPCN 4.754 09/12/34,-3.0,3.2,1.3,-27.0,-4.3,24.0,99.4,Airport,FIXED,GTAAIR,114.3,15.6,CAD,1318Z CN Equity,14.4,"1,122",A+,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,5.0,NBF,"3,000,000",NaN,NBF,"5,000,000",0,1,"3,367","3,625",39191ZAX7,92938WAD5
2117,GTAAIR 4.53 12/02/41,WSPCN 4.754 09/12/34,-4.5,3.1,-0.2,-28.7,-4.2,24.2,99.4,Airport,FIXED,GTAAIR,113.0,16.4,CAD,1318Z CN Equity,13.6,"1,098",A+,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,5.0,NBF,"3,000,000",NaN,NBF,"5,000,000",0,1,"3,294","3,625",39191ZAY5,92938WAD5
2285,ENBCN 6.1 11/09/32,PPLCN 3.53 12/10/31,11.2,3.1,12.4,-2.9,-1.1,14.1,99.4,Pipeline,FIXED,ENBCN,113.3,7.1,CAD,ENB CN Equity,2.7,637,BBB+,2-3,7.1-10.1,Midstream,FIXED,PPLCN,107.7,6.4,CAD,PPL CN Equity,3.6,541,BBB,3-5,5.1-7.1,5.0,RBC,"5,000,000",5.0,RBC,"5,000,000",1,1,"3,184","2,707",29251ZBW6,70632ZAS0
2498,BMO 5.039 05/29/28,BNS 4.68 02/01/29,-2.1,3.1,-2.1,-9.6,0.0,7.5,99.8,Bail In,FIXED,BMO,60.3,2.8,CAD,BMO CN Equity,2.1,271,A+,2-3,2.1-3.1,Bail In,FIXED,BNS,63.9,3.6,CAD,BNS CN Equity,1.4,338,A,1-2,3.1-4.1,3.0,TD,"15,000,000",3.0,BMO,"20,000,000",1,1,"4,061","6,756",06368LNK8,064164QM1
2707,MRUCN 3.39 12/06/27,WSPCN 4.754 09/12/34,-67.4,3.0,-62.6,-84.2,-4.9,16.8,99.4,Retail/Grocers,FIXED,MRUCN,51.1,2.1,CAD,MRU CN Equity,7.6,206,BBB,>7,2.1-3.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,4.0,RBC,"10,000,000",NaN,NBF,"5,000,000",0,1,"2,059","3,625",59162NAF6,92938WAD5
2783,ETRHWY 3.83 05/11/46,TCN 5 ¾ 09/08/33,-2.6,3.0,-2.6,-16.6,0.0,14.0,99.8,Infastructure,FIXED,ETRHWY,116.5,20.8,CAD,nan,10.2,"1,169",A,>7,10.1-25.1,Cable/Telco,FIXED,TCN,120.5,7.9,CAD,T CN Equity,1.8,687,BBB-,1-2,7.1-10.1,4.0,NBF,"5,000,000",5.0,RBC,"10,000,000",0,1,"5,843","6,867",35085ZBL9,87971MCC5
2840,BMO 5.039 05/29/28,TD 5.491 09/08/28,0.3,3.0,0.3,-6.5,0.0,6.9,99.8,Bail In,FIXED,BMO,60.3,2.8,CAD,BMO CN Equity,2.1,271,A+,2-3,2.1-3.1,Bail In,FIXED,TD,63.6,3.2,CAD,TD CN Equity,1.8,306,A,1-2,3.1-4.1,3.0,TD,"15,000,000",2.0,BMO,"20,000,000",1,1,"4,061","6,125",06368LNK8,89117GX51


In [14]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute At Least 1 Side

z_port_vs_universe_1way_liquidity = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_1way_liquidity.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
38,BMO 3.19 03/01/28,HOMEQU 6.552 10/18/27,-115.8,4.1,-115.8,-172.8,0.0,57.0,99.8,Dep Note,FIXED,BMO,43.8,2.6,CAD,BMO CN Equity,7.4,250,AA,>7,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,4.0,SCM,"10,000,000",NaN,CIBC,"3,000,000",0,1,"2,502",658,06368BTX6,43739CAB7
63,GTAAIR 4.53 12/02/41,HOMEQU 6.552 10/18/27,-46.5,4.0,-46.5,-102.1,0.0,55.5,99.8,Airport,FIXED,GTAAIR,113.0,16.4,CAD,1318Z CN Equity,13.6,"1,098",A+,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,NBF,"3,000,000",NaN,CIBC,"3,000,000",0,1,"3,294",658,39191ZAY5,43739CAB7
70,GTAAIR 2 ¾ 10/17/39,HOMEQU 6.552 10/18/27,-58.6,4.0,-58.6,-110.7,0.0,52.1,99.8,Airport,FIXED,GTAAIR,100.8,14.3,CAD,1318Z CN Equity,5.7,905,A+,5-7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,SCM,"3,000,000",NaN,CIBC,"3,000,000",0,1,"2,715",658,39191ZBD0,43739CAB7
79,GTAAIR 5.3 02/25/41,HOMEQU 6.552 10/18/27,-45.0,4.0,-45.0,-100.5,0.0,55.5,99.8,Airport,FIXED,GTAAIR,114.3,15.6,CAD,1318Z CN Equity,14.4,"1,122",A+,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,NBF,"3,000,000",NaN,CIBC,"3,000,000",0,1,"3,367",658,39191ZAX7,43739CAB7
97,FTSCN 3 ⅜ 04/13/45,HOMEQU 6.552 10/18/27,-42.3,3.9,-42.3,-96.5,0.0,54.2,99.8,Utility,FIXED,FTSCN,118.8,19.7,CAD,FTS CN Equity,10.3,"1,090",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,TD,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,180",658,34959ZAB8,43739CAB7
109,AERMON 3.919 09/26/42,HOMEQU 6.552 10/18/27,-46.7,3.9,-46.7,-97.8,0.0,51.2,99.8,Airport,FIXED,AERMON,114.2,17.2,CAD,58232Z CN Equity,12.8,"1,076",A+,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,NBF,"3,000,000",NaN,CIBC,"3,000,000",0,1,"3,227",658,007863AH6,43739CAB7
112,BRUPOW 4.7 06/21/31,HOMEQU 6.552 10/18/27,-71.7,3.9,-71.7,-120.4,0.0,48.7,99.8,Power Gen/Renewable,FIXED,BRUPOW,90.1,5.8,CAD,561679Z CN Equity,1.3,519,BBB,1-2,5.1-7.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,SCM,"5,000,000",NaN,CIBC,"3,000,000",0,1,"2,595",658,116705AN4,43739CAB7
114,ALTALK 4.09 06/30/45,HOMEQU 6.552 10/18/27,-45.7,3.9,-45.7,-98.1,0.0,52.5,99.8,Utility,FIXED,ALTALK,114.4,20.0,CAD,BRK/A US Equity,10.0,"1,179",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,1.0,NBF,"5,000,000",NaN,CIBC,"3,000,000",0,1,"5,894",658,02135ZAQ9,43739CAB7
116,ENBGAS 4 08/22/44,HOMEQU 6.552 10/18/27,-35.4,3.9,-35.4,-84.6,0.0,49.2,99.8,Utility,FIXED,ENBGAS,125.3,19.1,CAD,ENB CN Equity,10.9,"1,122",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,3.0,TD,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,243",658,29290ZAP5,43739CAB7
119,GTAAIR 5.63 06/07/40,HOMEQU 6.552 10/18/27,-42.9,3.9,-42.9,-100.4,0.0,57.5,99.8,Airport,FIXED,GTAAIR,116.4,14.9,CAD,1318Z CN Equity,15.1,"1,109",A+,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,5.0,NBF,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,218",658,39191ZAW9,43739CAB7


In [15]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute Both Sides At Reasonable Bid/Offer

z_port_vs_universe_2way_liquiditybidoffer = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['CUSIP_2 CR01 @ Best Bid'] >2000) &
    (z_all_enhanced['Bid/Offer_1'] <2) &
    (z_all_enhanced['Bid/Offer_2'] <2) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_2way_liquiditybidoffer.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2


In [16]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute At Least One Side At Reasonable Bid/Offer

z_port_vs_universe_1way_liquiditybidoffer = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['Bid/Offer_1'] <2) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_1way_liquiditybidoffer.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
114,ALTALK 4.09 06/30/45,HOMEQU 6.552 10/18/27,-45.7,3.9,-45.7,-98.1,0.0,52.5,99.8,Utility,FIXED,ALTALK,114.4,20.0,CAD,BRK/A US Equity,10.0,"1,179",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,1.0,NBF,"5,000,000",NaN,CIBC,"3,000,000",0,1,"5,894",658,02135ZAQ9,43739CAB7
135,ALTALK 4.054 11/21/44,HOMEQU 6.552 10/18/27,-45.1,3.9,-45.1,-96.7,0.0,51.7,99.8,Utility,FIXED,ALTALK,114.7,19.4,CAD,BRK/A US Equity,10.6,"1,156",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,1.0,NBF,"3,000,000",NaN,CIBC,"3,000,000",0,1,"3,468",658,02135ZAP1,43739CAB7
200,ENBGAS 4.88 06/21/41,HOMEQU 6.552 10/18/27,-32.1,3.8,-32.1,-84.3,0.0,52.2,99.8,Utility,FIXED,ENBGAS,128.2,15.9,CAD,ENB CN Equity,14.1,"1,083",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,0.0,NBF,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,165",658,90664ZAU4,43739CAB7
256,CUCN 4.851 06/03/52,HOMEQU 6.552 10/18/27,-37.4,3.7,-37.4,-82.9,0.0,45.5,99.8,Utility,FIXED,CUCN,122.7,26.9,CAD,ACO/X CN Equity,3.1,"1,430",A-,3-5,>25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,1.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,860",658,136717BD6,43739CAB7
301,ALTALK 4.462 11/08/41,HOMEQU 6.552 10/18/27,-40.4,3.7,-40.4,-94.5,0.0,54.1,99.8,Utility,FIXED,ALTALK,119.3,16.3,CAD,BRK/A US Equity,13.7,"1,080",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,1.0,NBF,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,159",658,02135ZAE6,43739CAB7
586,FTSCN 3.85 12/07/48,HOMEQU 6.552 10/18/27,-48.8,3.5,-48.8,-100.3,0.0,51.5,99.8,Utility,FIXED,FTSCN,112.0,23.4,CAD,FTS CN Equity,6.6,"1,235",A-,5-7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,-1.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,471",658,34959ZAH5,43739CAB7
611,ALCTRA 5.225 11/14/52,HOMEQU 6.552 10/18/27,-54.7,3.5,-54.7,-103.1,0.0,48.4,99.8,Utility,FIXED,ALCTRA,104.5,26.8,CAD,1487407D CN Equity,2.7,"1,539",A-,2-3,>25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,1.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"3,077",658,014443AM7,43739CAB7
619,ALTALK 5.381 03/26/40,HOMEQU 6.552 10/18/27,-36.8,3.5,-36.8,-93.3,0.0,56.4,99.8,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,-1.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,143",658,02135ZAC0,43739CAB7
678,BIP 5.95 07/27/53,HOMEQU 6.552 10/18/27,2.8,3.5,2.8,-39.4,0.0,42.3,99.8,Infastructure,FIXED,BIP,165.4,27.5,CAD,BIP US Equity,2.0,"1,472",BBB+,1-2,>25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,0.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,945",658,11291ZAN7,43739CAB7
1205,ALTALK 4.09 06/30/45,HOMEQU 7.108 12/11/26,-27.3,3.3,-25.5,-67.7,-1.8,40.3,97.4,Utility,FIXED,ALTALK,114.4,20.0,CAD,BRK/A US Equity,10.0,"1,179",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,1.0,NBF,"5,000,000",NaN,CIBC,"3,000,000",0,1,"5,894",418,02135ZAQ9,43739CAA9


In [17]:
# Bonds That May Look Cheap and We Can Execute At Least One Side At Reasonable Bid/Offer

z_cad_universe_cr01_bidoffer = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['Bid/Offer_1'] <1) 
].copy()

display(format_for_display(z_cad_universe_cr01_bidoffer.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
125,BCECN 2 ½ 05/14/30,RCICN 2.9 12/09/30,7.2,3.9,8.5,-14.4,-1.3,21.6,99.4,Cable/Telco,FIXED,BCECN,73.1,4.8,CAD,BCE CN Equity,5.2,423,BBB,5-7,4.1-5.1,Cable/Telco,FIXED,RCICN,71.1,5.4,CAD,RCI/B CN Equity,5.2,471,BBB-,5-7,5.1-7.1,-4.0,BMO,"5,000,000",5.0,RBC,"5,000,000",0,0,"2,117","2,356",07813ZCE2,775109CS8
185,FTSCN 3.85 12/07/48,LOWMAT 2.433 05/14/31,73.0,3.8,76.7,49.4,-3.7,23.6,99.0,Utility,FIXED,FTSCN,112.0,23.4,CAD,FTS CN Equity,6.6,"1,235",A-,5-7,10.1-25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,-1.0,BMO,"2,000,000",10.0,TD,"3,000,000",0,0,"2,471","1,503",34959ZAH5,548243AJ3
200,ENBGAS 4.88 06/21/41,HOMEQU 6.552 10/18/27,-32.1,3.8,-32.1,-84.3,0.0,52.2,99.8,Utility,FIXED,ENBGAS,128.2,15.9,CAD,ENB CN Equity,14.1,"1,083",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,0.0,NBF,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,165",658,90664ZAU4,43739CAB7
586,FTSCN 3.85 12/07/48,HOMEQU 6.552 10/18/27,-48.8,3.5,-48.8,-100.3,0.0,51.5,99.8,Utility,FIXED,FTSCN,112.0,23.4,CAD,FTS CN Equity,6.6,"1,235",A-,5-7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,-1.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,471",658,34959ZAH5,43739CAB7
619,ALTALK 5.381 03/26/40,HOMEQU 6.552 10/18/27,-36.8,3.5,-36.8,-93.3,0.0,56.4,99.8,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,-1.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,143",658,02135ZAC0,43739CAB7
649,ENBGAS 4.88 06/21/41,HYDONE 1.69 01/16/31,102.0,3.5,102.0,80.9,0.0,21.0,99.8,Utility,FIXED,ENBGAS,128.2,15.9,CAD,ENB CN Equity,14.1,"1,083",A-,>7,10.1-25.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,0.0,NBF,"2,000,000",NaN,RBC,"5,000,000",0,0,"2,165","2,359",90664ZAU4,44810ZCF5
678,BIP 5.95 07/27/53,HOMEQU 6.552 10/18/27,2.8,3.5,2.8,-39.4,0.0,42.3,99.8,Infastructure,FIXED,BIP,165.4,27.5,CAD,BIP US Equity,2.0,"1,472",BBB+,1-2,>25.1,Mortgage Provider,FIXED,HOMEQU,174.2,2.3,CAD,HEQ CN Equity,1.2,219,NR,1-2,2.1-3.1,0.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,945",658,11291ZAN7,43739CAB7
916,FTSCN 3.85 12/07/48,RCICN 2.9 12/09/30,48.2,3.4,49.0,13.0,-0.8,35.2,99.4,Utility,FIXED,FTSCN,112.0,23.4,CAD,FTS CN Equity,6.6,"1,235",A-,5-7,10.1-25.1,Cable/Telco,FIXED,RCICN,71.1,5.4,CAD,RCI/B CN Equity,5.2,471,BBB-,5-7,5.1-7.1,-1.0,BMO,"2,000,000",5.0,RBC,"5,000,000",0,0,"2,471","2,356",34959ZAH5,775109CS8
945,ENBGAS 4.88 06/21/41,GWOCN 3.337 02/28/28,94.6,3.4,94.6,73.4,0.0,21.1,99.8,Utility,FIXED,ENBGAS,128.2,15.9,CAD,ENB CN Equity,14.1,"1,083",A-,>7,10.1-25.1,Insurance Senior,FIXED,GWOCN,45.3,2.4,CAD,POW CN Equity,7.4,227,A,>7,2.1-3.1,0.0,NBF,"2,000,000",4.0,TD,"5,000,000",0,0,"2,165","1,134",90664ZAU4,39138CAH9
1043,FTSCN 3.85 12/07/48,T 4.85 05/25/47,-42.2,3.4,-42.0,-61.4,-0.3,19.2,99.4,Utility,FIXED,FTSCN,112.0,23.4,CAD,FTS CN Equity,6.6,"1,235",A-,5-7,10.1-25.1,Cable/Telco,FIXED,T,157.8,21.9,CAD,T US Equity,8.1,"1,220",BBB,>7,10.1-25.1,-1.0,BMO,"2,000,000",5.0,RBC,"2,000,000",0,0,"2,471","2,440",34959ZAH5,00206RDY5
